<a href="https://colab.research.google.com/github/buthaina279/DS_EDA_example/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
# Import Libraries

from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
from google.colab import drive  
#import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import itertools

In [2]:
# Connect Google Drive to Colab
drive.mount('/content/gdrive')
# Create a variable to store the data path on your drive
path = './gdrive/My Drive/datasets/twitter_analysis'

Mounted at /content/gdrive


In [88]:
saudia_airlines = pd.read_csv("./gdrive/My Drive/datasets/twitter_analysis/saudia_airlines_classified.csv")

In [89]:
saudia_airlines.head(10)

,Unnamed: 0,Unnamed: 0.1,tweet_id,created_at,text,likes,retweeted,location,language,airline,Tweet_nonstop,sentiment_confidence,polarity,sentiment
0,0,14,1335393841810202624,2020-12-06 01:21:16,dear guest there is no flight from india to s...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.900,0.600,Positive
1,1,23,1335337522738720770,2020-12-05 21:37:29,hello dear guest we hope you check and be inf...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'hope', 'check'...",0.375,-0.125,Negative
2,2,29,1335334320014315521,2020-12-05 21:24:45,hello dear guest please provide us with the b...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'please', 'prov...",0.400,0.100,Positive
3,3,38,1335322063851167748,2020-12-05 20:36:03,dear guest there is no flight from india to s...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
4,4,43,1335314759588122626,2020-12-05 20:07:02,hello dear guest you can follow up the flight...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'follow', 'flig...",0.000,0.000,Neutral
5,5,50,1335306576312356865,2020-12-05 19:34:31,dear guest there is no flight from india to s...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
6,6,51,1335304974679613443,2020-12-05 19:28:09,hello dear valued guest once the confirmation...,1,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'valued', 'guest', 'conf...",0.000,0.000,Neutral
7,7,53,1335276146653589507,2020-12-05 17:33:36,dear guest there is no flight from india to s...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
8,8,55,1335275872295804928,2020-12-05 17:32:30,dear guest there is no flight from india to s...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
9,9,57,1335266364739694593,2020-12-05 16:54:44,hello dear guest imran please provide us with...,0,False,Saudi Arabia,en,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'imran', 'pleas...",0.400,0.100,Positive


#preproccesing

#Delete uninformative variables

In [28]:
saudia_airlines = saudia_airlines.drop(['Unnamed: 0','Unnamed: 0.1','tweet_id', 'text', 'language'], axis=1)

In [32]:
saudia_airlines.head(10)

,created_at,likes,retweeted,location,airline,Tweet_nonstop,sentiment_confidence,polarity,sentiment
0,2020-12-06 01:21:16,1,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.900,0.600,Positive
1,2020-12-05 21:37:29,1,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'hope', 'check'...",0.375,-0.125,Negative
2,2020-12-05 21:24:45,0,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'please', 'prov...",0.400,0.100,Positive
3,2020-12-05 20:36:03,1,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
4,2020-12-05 20:07:02,1,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'follow', 'flig...",0.000,0.000,Neutral
5,2020-12-05 19:34:31,0,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
6,2020-12-05 19:28:09,1,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'valued', 'guest', 'conf...",0.000,0.000,Neutral
7,2020-12-05 17:33:36,0,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
8,2020-12-05 17:32:30,0,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
9,2020-12-05 16:54:44,0,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'imran', 'pleas...",0.400,0.100,Positive


#Devide the dataset into predictors and target

In [64]:
X = saudia_airlines.iloc[:,:-1].values #all predictor columns without the target
y = saudia_airlines.iloc[:,-1].values #last column (target)

In [66]:
saudia_airlines.head(10)

,likes,retweeted,location,airline,Tweet_nonstop,sentiment_confidence,polarity,sentiment
0,1,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.900,0.600,Positive
1,1,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'hope', 'check'...",0.375,-0.125,Negative
2,0,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'please', 'prov...",0.400,0.100,Positive
3,1,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
4,1,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'follow', 'flig...",0.000,0.000,Neutral
5,0,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
6,1,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'valued', 'guest', 'conf...",0.000,0.000,Neutral
7,0,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
8,0,False,Saudi Arabia,Saudi_airlines,"['', 'dear', 'guest', 'flight', 'india', 'saud...",0.000,0.000,Neutral
9,0,False,Saudi Arabia,Saudi_airlines,"['', 'hello', 'dear', 'guest', 'imran', 'pleas...",0.400,0.100,Positive


#Encoding the independent variables

- retweeted
- location
- airline

In [67]:
ct = ColumnTransformer(transformers=[('encoder',
                                      OneHotEncoder(), 
                                      [1,2,3])], 
                                      remainder='passthrough')

X = ct.fit_transform(X)
X = np.array(X)
print(X)


[[1.0 1.0 1.0 ...
  "['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'kindly', 'follow', 'flight', '']"
  0.9 0.6]
 [1.0 1.0 1.0 ...
  "['', 'hello', 'dear', 'guest', 'hope', 'check', 'informed', 'travel', 'update', 'due', 'corona', 'virus', 'covid', '']"
  0.375 -0.125]
 [1.0 1.0 1.0 ...
  "['', 'hello', 'dear', 'guest', 'please', 'provide', 'u', 'booking', 'number', 'direct', 'message', 'let', 'know', 'thank']"
  0.4 0.1]
 ...
 [1.0 1.0 1.0 ...
  "['', 'dear', 'guest', 'kindly', 'check', 'flight', 'schedule', 'choose', 'flight', 'info', '']"
  0.9 0.6]
 [1.0 1.0 1.0 ...
  "['', 'flight', 'resumed', 'show', 'website', 'book', 'flight', 'resident', 'ex', '']"
  0.0 0.0]
 [1.0 1.0 1.0 ...
  "['', 'flight', 'resumed', 'show', 'website', 'yo', 'book', 'flight', 'resident', 'e', '']"
  0.0 0.0]]


#Encoding the dependent variables


In [68]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[2 0 2 ... 2 1 1]


Dealing with datetime

In [61]:
from datetime import datetime
for date in saudia_airlines.iloc[:,0]:
  datetime_object = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

In [62]:
saudia_airlines.iloc[:,0]

0       2020-12-06 01:21:16
1       2020-12-05 21:37:29
2       2020-12-05 21:24:45
3       2020-12-05 20:36:03
4       2020-12-05 20:07:02
               ...         
1459    2020-10-25 13:13:11
1460    2020-10-25 13:13:05
1461    2020-10-25 13:13:00
1462    2020-10-25 12:18:46
1463    2020-10-25 12:13:41
Name: created_at, Length: 1464, dtype: object

for now I will delete and next time I'll add it and see if it can affect the classification

In [63]:
saudia_airlines = saudia_airlines.drop(['created_at'], axis=1)

In [93]:
saudia_airlines['Tweet_nonstop']

0       ['', 'dear', 'guest', 'flight', 'india', 'saud...
1       ['', 'hello', 'dear', 'guest', 'hope', 'check'...
2       ['', 'hello', 'dear', 'guest', 'please', 'prov...
3       ['', 'dear', 'guest', 'flight', 'india', 'saud...
4       ['', 'hello', 'dear', 'guest', 'follow', 'flig...
                              ...                        
1459    ['', 'dear', 'guest', 'kindly', 'check', 'flig...
1460    ['', 'dear', 'guest', 'kindly', 'check', 'flig...
1461    ['', 'dear', 'guest', 'kindly', 'check', 'flig...
1462    ['', 'flight', 'resumed', 'show', 'website', '...
1463    ['', 'flight', 'resumed', 'show', 'website', '...
Name: Tweet_nonstop, Length: 1464, dtype: object

#Extracting Features from Cleaned Tweets

https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/



CounterVectorizer method needs a list of strings and not tokens.*italicized text*

In [166]:
def join_tokens(tokens):
  document = "".join([word for word in tokens if not word.isdigit()])
  return document

saudia_airlines['nonstop_documents'] = saudia_airlines['Tweet_nonstop'].apply(lambda x: join_tokens(x))


In [167]:
saudia_airlines['Tweet_nonstop'].head(3)

0    ['', 'dear', 'guest', 'flight', 'india', 'saud...
1    ['', 'hello', 'dear', 'guest', 'hope', 'check'...
2    ['', 'hello', 'dear', 'guest', 'please', 'prov...
Name: Tweet_nonstop, dtype: object

In [168]:
def join_tokens(tokens):
  document = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
  return document

saudia_airlines['nonstop_documents'] = saudia_airlines['Tweet_nonstop'].apply(lambda x: join_tokens(x))

In [169]:
saudia_airlines['nonstop_documents'].head(3)

0    ['',  ' d e a r',  ' g u e s t',  ' f l i g h ...
1    ['',  ' h e l l o',  ' d e a r',  ' g u e s t'...
2    ['',  ' h e l l o',  ' d e a r',  ' g u e s t'...
Name: nonstop_documents, dtype: object

In [170]:
from nltk.tokenize.treebank import TreebankWordDetokenizer



In [171]:
#for tweet in saudia_airlines['Tweet_nonstop']:
TreebankWordDetokenizer().detokenize(saudia_airlines['Tweet_nonstop'])
#print(tweet)

"['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'kindly', 'follow', 'flight', ''] ['', 'hello', 'dear', 'guest', 'hope', 'check', 'informed', 'travel', 'update', 'due', 'corona', 'virus', 'covid', ''] ['', 'hello', 'dear', 'guest', 'please', 'provide', 'u', 'booking', 'number', 'direct', 'message', 'let', 'know', 'thank'] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'hello', 'dear', 'guest', 'follow', 'flight', 'schedule', 'website', ''] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'hello', 'dear', 'valued', 'guest', 'confirmation', 'date', 'lifting', 'restriction', 'international', 'fl', ''] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'hello', 'dear', 'guest', 'imran', 'please', '

In [220]:
saudia_airlines.iloc[:,10].iloc[0][1:6]

"'', '"

In [211]:
saudia_airlines.iloc[:,10].dtypes

dtype('O')

In [177]:
TreebankWordDetokenizer().detokenize(ss)

"[' ',  ' d e a r',  ' g u e s t',  ' f l i g h t',  ' i n d i a',  ' s a u d i',  ' a r a b i a',  ' t i m e',  ' k i n d l y',  ' f o l l o w',  ' f l i g h t',  ' ' ]"

In [195]:
saudia_airlines['Tweet_nonstop'][1]

"['', 'hello', 'dear', 'guest', 'hope', 'check', 'informed', 'travel', 'update', 'due', 'corona', 'virus', 'covid', '']"

In [193]:
''.join(saudia_airlines['Tweet_nonstop'][0])

"['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'kindly', 'follow', 'flight', '']"

In [200]:
''.join(''.join(TreebankWordDetokenizer().detokenize(saudia_airlines['Tweet_nonstop'])))

"['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'kindly', 'follow', 'flight', ''] ['', 'hello', 'dear', 'guest', 'hope', 'check', 'informed', 'travel', 'update', 'due', 'corona', 'virus', 'covid', ''] ['', 'hello', 'dear', 'guest', 'please', 'provide', 'u', 'booking', 'number', 'direct', 'message', 'let', 'know', 'thank'] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'hello', 'dear', 'guest', 'follow', 'flight', 'schedule', 'website', ''] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'hello', 'dear', 'valued', 'guest', 'confirmation', 'date', 'lifting', 'restriction', 'international', 'fl', ''] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'flight', 'schedule', 'back', ''] ['', 'hello', 'dear', 'guest', 'imran', 'please', '

In [179]:
flattened_list = []

#flatten the lis
for x in saudia_airlines['Tweet_nonstop']:
    for y in x:
        flattened_list.append(y)

print(flattened_list)

['[', "'", "'", ',', ' ', "'", 'd', 'e', 'a', 'r', "'", ',', ' ', "'", 'g', 'u', 'e', 's', 't', "'", ',', ' ', "'", 'f', 'l', 'i', 'g', 'h', 't', "'", ',', ' ', "'", 'i', 'n', 'd', 'i', 'a', "'", ',', ' ', "'", 's', 'a', 'u', 'd', 'i', "'", ',', ' ', "'", 'a', 'r', 'a', 'b', 'i', 'a', "'", ',', ' ', "'", 't', 'i', 'm', 'e', "'", ',', ' ', "'", 'k', 'i', 'n', 'd', 'l', 'y', "'", ',', ' ', "'", 'f', 'o', 'l', 'l', 'o', 'w', "'", ',', ' ', "'", 'f', 'l', 'i', 'g', 'h', 't', "'", ',', ' ', "'", "'", ']', '[', "'", "'", ',', ' ', "'", 'h', 'e', 'l', 'l', 'o', "'", ',', ' ', "'", 'd', 'e', 'a', 'r', "'", ',', ' ', "'", 'g', 'u', 'e', 's', 't', "'", ',', ' ', "'", 'h', 'o', 'p', 'e', "'", ',', ' ', "'", 'c', 'h', 'e', 'c', 'k', "'", ',', ' ', "'", 'i', 'n', 'f', 'o', 'r', 'm', 'e', 'd', "'", ',', ' ', "'", 't', 'r', 'a', 'v', 'e', 'l', "'", ',', ' ', "'", 'u', 'p', 'd', 'a', 't', 'e', "'", ',', ' ', "'", 'd', 'u', 'e', "'", ',', ' ', "'", 'c', 'o', 'r', 'o', 'n', 'a', "'", ',', ' ', "'", 'v',

In [173]:
TreebankWordDetokenizer().detokenize(['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'kindly', 'follow', 'flight', ''])

'dear guest flight india saudi arabia time kindly follow flight'

In [196]:
TreebankWordDetokenizer().detokenize(saudia_airlines['Tweet_nonstop'][0])

"[' ',  ' d e a r',  ' g u e s t',  ' f l i g h t',  ' i n d i a',  ' s a u d i',  ' a r a b i a',  ' t i m e',  ' k i n d l y',  ' f o l l o w',  ' f l i g h t',  ' ' ]"

In [ ]:
saudia_airlines.head(3)

In [115]:
ss = saudia_airlines.iloc[0,10]
ss

"['', 'dear', 'guest', 'flight', 'india', 'saudi', 'arabia', 'time', 'kindly', 'follow', 'flight', '']"

In [ ]:
saudia_airlines['nonstop_documents'].head()

#Bag-of-Words Features

Bag-of-Words is a method to represent text into numerical features


In [69]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(saudia_airlines['Tweet_nonstop'])

In [ ]:
bow

#TF-IDF Features

This is another method which is based on the frequency method but it is different to the bag-of-words approach in the sense that it takes into account, not just the occurrence of a word in a single document (or tweet) but in the entire corpus.

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(saudia_airlines['Tweet_nonstop'])

#Model calibration

https://towardsdatascience.com/classifier-calibration-7d0be1e05452

In [77]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=123)

Next, we instantiate two classifiers to compare; a simple logistic regression model and an implementation of Support Vector Machines provided by scikit-learn.

In [72]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


lgr = LogisticRegression(C=1, solver='lbfgs')
svc = SVC(max_iter=10000, probability=True)

Finally, we fit the classifiers to our training data and compute our predictions on the test data set. Specifically for the SVM, to get the probabilities for the positive class we need to know how the decision function separates the test samples, and normalize the results to be between `0` and `1`.

In [78]:
#probs_lgr = lgr.fit(X_train, y_train).predict_proba(X_test)[:,-1]
preds_svc = svc.fit(X_train, y_train)



ValueError: ignored

In [ ]:
.predict(X_test)


In [ ]:
probs_svc = svc.decision_function(X_test)
probs_svc = (probs_svc - probs_svc.min()) / (probs_svc.max() - probs_svc.min())

Let us now plot the Kernel Density Estimation for the two classifiers.

In [ ]:
plt.figure(figsize=(10,6))
sns.kdeplot(probs_lgr, label='Logistic regression')
sns.kdeplot(preds_svc, label='SVM')
plt.title("Probability Density Plot for 2 Classifiers")
plt.show()

let us check the AUC-ROC curve for the two binary classifiers, but this time using the probabilities we calculated for the SVM.

In [ ]:
from sklearn import metrics


plt.figure(figsize=(8,6))
plt.plot([0, 1], [0, 1],'r--')

pred = probs_lgr
label = y_test
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr, tpr, label=f'Logistic regression, auc = {str(round(auc,3))}')

pred = probs_svc
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr, tpr, label=f'SVC, auc = {str(round(auc,3))}')

plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.title("AUC-ROC for two models")
plt.legend()
plt.show()

To plot the calibration curve of each classifier we define a utility function like the one below.


In [ ]:
from sklearn.calibration import calibration_curve


def plot_calibration_curve(name, fig_index, probs):
    """Plot calibration curve for est w/o and with calibration. """

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))
    
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    
    frac_of_pos, mean_pred_value = calibration_curve(y_test, probs, n_bins=10)

    ax1.plot(mean_pred_value, frac_of_pos, "s-", label=f'{name}')
    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title(f'Calibration plot ({name})')
    
    ax2.hist(probs, range=(0, 1), bins=10, label=name, histtype="step", lw=2)
    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")

In [ ]:
# plot calibration curve for logistic regression
plot_calibration_curve("Logistic regression", 1, probs_lgr)

In [ ]:
# plot calibration curve for the SVM
plot_calibration_curve("SVM", 1, probs_svc)

#Calibrating the model

The two most popular methods of calibrating a machine learning model are the isotonic and Platt's method.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV


lgr = LogisticRegression(C=1, solver='lbfgs')
svc = SVC(max_iter=10000, probability=True)

platts_scaling = CalibratedClassifierCV(svc, cv=2, method='sigmoid')
platts_scaling.fit(X_train, y_train)
calibrated_probs = platts_scaling.predict_proba(X_test)[:,1]

plot_calibration_curve("SVM", 3, calibrated_probs)